# Representation Learning at the Houses of the Oireachtas in Ireland




Notebook dedicated to run representation learning on the Houses of the Oireachtas' data in Ireland.

This work is based on a Senator Representations' work of the US Congress by Nathaniel Tucker:

https://github.com/knathanieltucker/tf-keras-tutorial/blob/master/SenatorRepresentations.ipynb

The Oireachtas is the legislature of the Republic of Ireland.

The Oireachtas consists of:
- The President of Ireland
- The two houses of the Oireachtas:
    - Dáil Éireann (lower house)
    - Seanad Éireann (upper house)
    
Further info: https://en.wikipedia.org/wiki/Oireachtas

Information regarding the Houses of the Oireachtas is featured on their website, including legislation, and is the copyright of the Houses of the Oireachtas: https://beta.oireachtas.ie/. 

Another interesting resource is https://www.kildarestreet.com/ which is a searchable archive of everything that's been said in the Dáil and all written parliamentary questions since January 2004, everything in the Seanad since September 2002, and all Committee meetings since September 2012.

Open Data from the Houses of the Oireachtas can be accessed via:
https://beta.oireachtas.ie/en/open-data/

This includes a link to the open data APIs via a Swagger UI:
https://api.oireachtas.ie/

A vote in the Houses of the Oireachtas is also called a division. We will be looking at this divisions (votings) table for the house and the Seanad (Senate):

In [1]:
import requests # http://docs.python-requests.org/
# from pattern import web
from BeautifulSoup import BeautifulSoup
import json

In [2]:
def get_votings():
    
    votings_data = []
    
    table = "divisions" # votings table
    url = 'https://api.oireachtas.ie/v1/{}'.format(table)
    date_start = '1900-01-01'
    date_end = '2017-12-31'
    chamber_type = 'house' # 'committee' or 'house'
    chamber = 'seanad' # 'seanad' or 'dail'
    
    batch = 500
    skip = 0
    limit = batch
    count = 1 # start
    
    while count > 0:
        
        params = dict(chamber_type=chamber_type,
                      chamber=chamber, 
                      date_start=date_start, 
                      date_end=date_end,
                      skip=skip, 
                      limit=limit)
        r = requests.get(url, params=params)
        
        print r.url
        # print "Status Code:", r.status_code
        # print "Headers:", r.headers
        
        # Add data from this batch
        contents = r.json()
        votings = contents["results"][:batch] # in order no to have duplicates
        votings_data.extend(votings)

        # Update count, more to retrieve?
        count = len(votings)
        print "Number retrieved", count
        # Update skip and limit for query
        skip += batch
        limit += batch
        
    return votings_data

In [3]:
# all_vote_data = get_votings()

In [4]:
# with open('data/congress/Ireland/all_vote_data.json', 'w') as outfile:
#      json.dump(all_vote_data, outfile)

In [5]:
all_vote_data = json.load(open('data/congress/Ireland/all_vote_data.json'))

How many votings?

In [6]:
len(all_vote_data)

3516

Verify there is no duplicates:

In [7]:
def verify_no_voting_duplicates(vote_data):
    votes = []
    duplicates = []
    for vote in vote_data:
        uri = vote["division"]["uri"]
        if uri not in votes:
            # print uri
            votes.append(uri)
        else:
            duplicates.append(uri)
    return len(duplicates)

In [8]:
verify_no_voting_duplicates(all_vote_data)

0

Get senators:

In [9]:
def get_senators(vote_data):
    senators = []
    for vote in vote_data:
        div = vote["division"]
        for outcome, voting in div["tallies"].iteritems():            
            if voting is not None:
                # print voting["showAs"] # Tá (YES), Níl (NO), Staon (Abstention)
                # print len(voting["members"])
                for member in voting["members"]:
                    member = member["member"]
                    #last_name, first_name = member["showAs"].split(",")
                    #first_name = first_name[:-1].strip()
                    #last_name = last_name.strip()
                    member_code = member["memberCode"]
                    # print member_code
                    if member_code is not None:
                        senators.append(member_code)
    return senators

In [10]:
senators = get_senators(all_vote_data)

In [11]:
senators[3]

u'Jerry-Buttimer.S.2007-07-23'

In [12]:
for v, k in enumerate(set(senators)):
    print v, k

0 Cecilia-Keaveney.D.1996-04-02
1 Déirdre-de-Búrca.S.2007-07-23
2 Alex-White.S.2007-07-23
3 Maria-Byrne.S.2016-04-25
4 Colm-Burke.S.2011-05-25
5 Katherine-Zappone.S.2011-05-25
6 Billy-Lawless.S.2016-04-25
7 Helen-McEntee.D.2013-03-27
8 Aodhán-Ó-Ríordáin.D.2011-03-09
9 Dan-Boyle.D.2002-06-06
10 Gerald-Nash.D.2011-03-09
11 Pat-Moylan.S.1997-09-17
12 John-Halligan.D.2011-03-09
13 Pat-O'Neill.S.2011-05-25
14 Kieran-Phelan.S.2002-09-12
15 Martin-Kenny.D.2016-10-03
16 John-Gilroy.S.2011-05-25
17 Fergal-Browne.S.2002-09-12
18 Colette-Kelleher.S.2016-04-25
19 Neale-Richmond.S.2016-04-25
20 Kate-Walsh.S.2002-09-12
21 John-Whelan.S.2011-05-25
22 John-Browne.D.1982-12-14
23 Frank-Feighan.S.2002-09-12
24 Pearse-Doherty.S.2007-07-23
25 Mary-Ann-O'Brien.S.2011-05-25
26 Jerry-Buttimer.S.2007-07-23
27 Derek-McDowell.D.1992-12-14
28 Niall-Ó-Brolcháin.S.2009-12-14
29 Cyprian-Brady.S.2002-09-12
30 Paschal-Donohoe.S.2007-07-23
31 Alice-Mary-Higgins.S.2016-04-25
32 Paul-Daly.S.2016-04-25
33 Peter-Callanan.

In [13]:
# leave the first two blank for padding and not senators
senator_to_id = { k: v + 2 for v, k in enumerate(set(senators)) }

In [14]:
senator_to_id

{u'Aidan-Davitt.S.2016-04-25': 60,
 u'Aideen-Hayden.S.2011-05-25': 133,
 u'Alan-Kelly.S.2007-07-23': 67,
 u'Alex-White.S.2007-07-23': 4,
 u'Alice-Mary-Higgins.S.2016-04-25': 33,
 u'Ann-Ormonde.S.1993-02-17': 192,
 u'Aodh\xe1n-\xd3-R\xedord\xe1in.D.2011-03-09': 10,
 u'Averil-Power.S.2011-05-25': 153,
 u'Billy-Lawless.S.2016-04-25': 8,
 u'Brendan-Daly.D.1973-03-14': 127,
 u'Brendan-Kenneally.D.1989-06-29': 76,
 u'Brendan-Ryan.S.1982-05-13': 156,
 u'Brendan-Ryan.S.2007-07-23': 185,
 u'Brian-Hayes.S.1995-12-20': 74,
 u'Brian-\xd3-Domhnaill.S.2007-07-23': 148,
 u'Camillus-Glynn.S.1997-09-17': 84,
 u'Catherine-Ardagh.S.2016-04-25': 85,
 u'Catherine-Noone.S.2011-05-25': 114,
 u'Cecilia-Keaveney.D.1996-04-02': 2,
 u'Chris-Wall.S.2007-06-23': 57,
 u'Ciaran-Cannon.S.2007-07-23': 42,
 u'Colette-Kelleher.S.2016-04-25': 20,
 u'Colm-Burke.S.2011-05-25': 6,
 u'Cyprian-Brady.S.2002-09-12': 31,
 u'C\xe1it-Keane.S.2011-05-25': 56,
 u'Dan-Boyle.D.2002-06-06': 11,
 u"Darragh-O'Brien.D.2007-06-14": 37,
 u'

In [15]:
def get_senator_id(name):
    name = name.replace(" ", "-").replace(".", "")
    sens = [k for k, v in senator_to_id.iteritems() if k is not None and k.startswith(name)]
    if len(sens) > 0:
        return senator_to_id[sens[0]]
    return 1

In [16]:
get_senator_id('Aideen Hayden')

133

In [17]:
def get_senator_party(name):
    page = requests.get('https://beta.oireachtas.ie/en/members/member/%s/' % (name)).text
    # dom = web.Element(page)
    # party = dom.content.split('<p class="bio-text">Party:</p> <p>')[1].split('</p')[0].strip()
    bs = BeautifulSoup(page)
    party = bs.find(text="Party:").findNext('p').contents[0].strip()
    return party

In [18]:
senators[0], get_senator_party(senators[0])

(u'Colm-Burke.S.2011-05-25', u'Fine Gael')

In [19]:
senators[17], get_senator_party(senators[17])

(u'R\xf3n\xe1n-Mullen.S.2007-07-23', u'Independent')

In [20]:
parties = {}
for senator in set(senators):
    parties[senator] = get_senator_party(senator)

In [21]:
parties

{u'Aidan-Davitt.S.2016-04-25': u'Fianna F\xe1il',
 u'Aideen-Hayden.S.2011-05-25': u'Labour Party',
 u'Alan-Kelly.S.2007-07-23': u'Labour',
 u'Alex-White.S.2007-07-23': u'Labour Party',
 u'Alice-Mary-Higgins.S.2016-04-25': u'Independent',
 u'Ann-Ormonde.S.1993-02-17': u'Fianna F\xe1il',
 u'Aodh\xe1n-\xd3-R\xedord\xe1in.D.2011-03-09': u'Labour',
 u'Averil-Power.S.2011-05-25': u'Fianna F\xe1il',
 u'Billy-Lawless.S.2016-04-25': u'Independent',
 u'Brendan-Daly.D.1973-03-14': u'Fianna F\xe1il',
 u'Brendan-Kenneally.D.1989-06-29': u'Fianna F\xe1il',
 u'Brendan-Ryan.S.1982-05-13': u'Labour Party',
 u'Brendan-Ryan.S.2007-07-23': u'Labour',
 u'Brian-Hayes.S.1995-12-20': u'Fine Gael',
 u'Brian-\xd3-Domhnaill.S.2007-07-23': u'Fianna F\xe1il',
 u'Camillus-Glynn.S.1997-09-17': u'Fianna F\xe1il',
 u'Catherine-Ardagh.S.2016-04-25': u'Fianna F\xe1il',
 u'Catherine-Noone.S.2011-05-25': u'Fine Gael',
 u'Cecilia-Keaveney.D.1996-04-02': u'Fianna F\xe1il',
 u'Chris-Wall.S.2007-06-23': u'Fianna F\xe1il',
 u'

In [22]:
set(val for val in parties.values())

{u'Fianna F\xe1il',
 u'Fine Gael',
 u'Green Party',
 u'Independent',
 u'Labour',
 u'Labour Party',
 u'Progressive Democrats',
 u'Sinn F\xe9in'}

Extract sponsors and members against a bill based on a line of text manually imputer and with several formats:

In [23]:
# Sponsors and members against a Bill are specified in a text line where we will extract:
# 2 members that sponsored the bill
# 2 members that opposed to the bill
def get_sponsors(tellers_data):
    
    #print tellers_data
    
    sp_first, sp_second, ag_first, ag_second = None, None, None, None
    
    if ";" in tellers_data:
        
        tellers = tellers_data.split(";")
        
        for teller in tellers:
            
            if teller.strip() == "":
                continue
            if "Tellers:" in teller:
                teller = teller.split("Tellers:")[1].strip()
            elif ":" in teller:
                teller = teller.split(":")[1].strip()

            if "," in teller:
                values = teller.split(",")
            elif ":" in teller:
                values = teller.split(":")
            else:
                #print "Bad format"
                continue
            if len(values) != 2:
                #print "BAD format"
                continue
                
            support, senators = values

            if "Senators" in senators:
                senators = senators.split("Senators")
                senators = senators[-1]
                
            if " and" in senators:
                senators = senators.split(" and")
            elif "and " in senators:
                senators = senators.split("and ")
            elif "an d":
                senators = senators.split("an d")
            else:
                senators = senators.split("and")
            
            first_senator, second_senator = senators[:2]
            first_senator = first_senator.strip()
            second_senator = second_senator.strip()
            support = support.strip()

            if support.encode('utf-8') == str('T\xc3\xa1'):
                sp_first = get_senator_id(first_senator)
                sp_second = get_senator_id(second_senator)
                #print "Sponsors", first_senator, sp_first, second_senator, sp_second
            elif support.encode('utf-8') == str('N\xc3\xadl'):
                ag_first = get_senator_id(first_senator)
                ag_second = get_senator_id(second_senator)
                #print "Against", first_senator, ag_first, second_senator, ag_second
                
    return sp_first, sp_second, ag_first, ag_second 

In [24]:
# Not ALL senators will appear on the votings 
id_to_displayname = {}
for member_code in set(senators):
    senator_id = senator_to_id[member_code]
    name = member_code
    party = parties[name]
    if name is not None and '.' in name:
        name = name.split('.')[0].split('-')[-1] # last name
    if party is not None:
        name = name + ", " + party
    id_to_displayname[senator_id] = name

In [25]:
id_to_displayname

{2: u'Keaveney, Fianna F\xe1il',
 3: u'B\xfarca, Green Party',
 4: u'White, Labour Party',
 5: u'Byrne, Fine Gael',
 6: u'Burke, Fine Gael',
 7: u'Zappone, Independent',
 8: u'Lawless, Independent',
 9: u'McEntee, Fine Gael',
 10: u'R\xedord\xe1in, Labour',
 11: u'Boyle, Green Party',
 12: u'Nash, Labour',
 13: u'Moylan, Fianna F\xe1il',
 14: u'Halligan, Independent',
 15: u"O'Neill, Fine Gael",
 16: u'Phelan, Fianna F\xe1il',
 17: u'Kenny, Sinn F\xe9in',
 18: u'Gilroy, Labour Party',
 19: u'Browne, Fine Gael',
 20: u'Kelleher, Independent',
 21: u'Richmond, Fine Gael',
 22: u'Walsh, Progressive Democrats',
 23: u'Whelan, Labour Party',
 24: u'Browne, Fianna F\xe1il',
 25: u'Feighan, Fine Gael',
 26: u'Doherty, Sinn F\xe9in',
 27: u"O'Brien, Independent",
 28: u'Buttimer, Fine Gael',
 29: u'McDowell, Labour Party',
 30: u'Brolch\xe1in, Green Party',
 31: u'Brady, Fianna F\xe1il',
 32: u'Donohoe, Fine Gael',
 33: u'Higgins, Independent',
 34: u'Daly, Fianna F\xe1il',
 35: u'Callanan, Fi

In [26]:
senator_vote_data = []

for vote in all_vote_data:
    
    div = vote["division"]
    #outcome = div["outcome"] # Carried, Lost, _
    #print div["date"]
    #print div["isBill"]
    #print div["category"]
    #print div["chamber"]["showAs"]
    #print div["debate"]["showAs"]
    
    # Bill sponsors
    sponsors = get_sponsors(div["tellers"])
    
    if None not in sponsors:
        
        sponsor, cosponsor, against, coagainst = sponsors
        
        tallies = div["tallies"]
        
        if tallies["taVotes"] is not None:
            for member in tallies["taVotes"]["members"]:
                member = member["member"]
                name = member["showAs"]
                last_name = name.split(',')[0]
                #uri = member["uri"]
                #short = member["memberCode"]
                if member["memberCode"] is not None:
                    senator_id = senator_to_id[member["memberCode"]]
                    party = parties[member["memberCode"]]
                    id_to_displayname[senator_id] = last_name + ", " + party # replace names with utf8 encoding names
                    senator_vote_data.append((1, senator_id, sponsor, [cosponsor, against, coagainst])) 
                
        elif tallies["nilVotes"] is not None:
            for member in tallies["nilVotes"]["members"]:
                member = member["member"]
                name = member["showAs"]
                last_name = name.split(',')[0]
                if member["memberCode"] is not None:
                    senator_id = senator_to_id[member["memberCode"]]
                    party = parties[member["memberCode"]]
                    id_to_displayname[senator_id] = last_name + ", " + party
                    senator_vote_data.append((0, senator_id, sponsor, [cosponsor, against, coagainst]))     

In [27]:
senator_vote_data

[(1, 6, 190, [117, 101, 93]),
 (1, 122, 190, [117, 101, 93]),
 (1, 158, 190, [117, 101, 93]),
 (1, 28, 190, [117, 101, 93]),
 (1, 5, 190, [117, 101, 93]),
 (1, 87, 190, [117, 101, 93]),
 (1, 146, 190, [117, 101, 93]),
 (1, 34, 190, [117, 101, 93]),
 (1, 60, 190, [117, 101, 93]),
 (1, 25, 190, [117, 101, 93]),
 (1, 91, 190, [117, 101, 93]),
 (1, 162, 190, [117, 101, 93]),
 (1, 195, 190, [117, 101, 93]),
 (1, 20, 190, [117, 101, 93]),
 (1, 129, 190, [117, 101, 93]),
 (1, 190, 190, [117, 101, 93]),
 (1, 139, 190, [117, 101, 93]),
 (1, 81, 190, [117, 101, 93]),
 (1, 39, 190, [117, 101, 93]),
 (1, 165, 190, [117, 101, 93]),
 (1, 72, 190, [117, 101, 93]),
 (1, 144, 190, [117, 101, 93]),
 (1, 117, 190, [117, 101, 93]),
 (1, 99, 190, [117, 101, 93]),
 (1, 167, 190, [117, 101, 93]),
 (1, 21, 190, [117, 101, 93]),
 (1, 176, 190, [117, 101, 93]),
 (1, 73, 190, [117, 101, 93]),
 (1, 85, 85, [164, 190, 117]),
 (1, 157, 85, [164, 190, 117]),
 (1, 164, 85, [164, 190, 117]),
 (1, 104, 85, [164, 190, 1

In [28]:
id_to_displayname

{2: u'Keaveney, Fianna F\xe1il',
 3: u'de B\xfarca, Green Party',
 4: u'White, Labour Party',
 5: u'Byrne, Fine Gael',
 6: u'Burke, Fine Gael',
 7: u'Zappone, Independent',
 8: u'Lawless, Independent',
 9: u'McEntee, Fine Gael',
 10: u'\xd3 R\xedord\xe1in, Labour',
 11: u'Boyle, Green Party',
 12: u'Craughwell, Labour',
 13: u'Moylan, Fianna F\xe1il',
 14: u'Halligan, Independent',
 15: u'O\u2019Neill, Fine Gael',
 16: u'Phelan, Fianna F\xe1il',
 17: u"O'Brien, Sinn F\xe9in",
 18: u'Gilroy, Labour Party',
 19: u'Browne, Fine Gael',
 20: u'Kelleher, Independent',
 21: u'Richmond, Fine Gael',
 22: u'Walsh, Progressive Democrats',
 23: u'Whelan, Labour Party',
 24: u'Crown, Fianna F\xe1il',
 25: u'Feighan, Fine Gael',
 26: u'Doherty, Sinn F\xe9in',
 27: u'O\u2019Brien, Independent',
 28: u'Buttimer, Fine Gael',
 29: u'McDowell, Labour Party',
 30: u'\xd3 Brolch\xe1in, Green Party',
 31: u'Brady, Fianna F\xe1il',
 32: u'Donohoe, Fine Gael',
 33: u'Higgins, Independent',
 34: u'Daly, Fianna

In [29]:
len(senator_vote_data)

38471

~40 examples of (vote, senator voting, sponsor, cosponsor, against, coagainst) tuples is pretty good (we could of course scrape more).

In [30]:
y = [d[0] for d in senator_vote_data]

In [31]:
len(y)

38471

In [32]:
# again we pad
def pad_or_crop(lst, l=10):
    return (lst + [0] * l)[:10]

In [33]:
import numpy as np

x_1 = np.array(map(lambda x: x[1], senator_vote_data))
x_2 = np.array(map(lambda x: x[2], senator_vote_data))
x_3 = np.array(map(lambda x: pad_or_crop(x[3]), senator_vote_data))
x = [x_1, x_2, x_3]

In [34]:
x

[array([  6, 122, 158, ..., 177,  22,  73]),
 array([190, 190, 190, ...,   1,   1,   1]),
 array([[117, 101,  93, ...,   0,   0,   0],
        [117, 101,  93, ...,   0,   0,   0],
        [117, 101,  93, ...,   0,   0,   0],
        ..., 
        [  1,   1,   1, ...,   0,   0,   0],
        [  1,   1,   1, ...,   0,   0,   0],
        [  1,   1,   1, ...,   0,   0,   0]])]

In [35]:
len(x)

3

In [36]:
# we add in padding and unknown senators
id_to_displayname[0] = '<PAD>'
id_to_displayname[1] = '<NOT A SENATOR>'

In [37]:
id_to_displayname

{0: '<PAD>',
 1: '<NOT A SENATOR>',
 2: u'Keaveney, Fianna F\xe1il',
 3: u'de B\xfarca, Green Party',
 4: u'White, Labour Party',
 5: u'Byrne, Fine Gael',
 6: u'Burke, Fine Gael',
 7: u'Zappone, Independent',
 8: u'Lawless, Independent',
 9: u'McEntee, Fine Gael',
 10: u'\xd3 R\xedord\xe1in, Labour',
 11: u'Boyle, Green Party',
 12: u'Craughwell, Labour',
 13: u'Moylan, Fianna F\xe1il',
 14: u'Halligan, Independent',
 15: u'O\u2019Neill, Fine Gael',
 16: u'Phelan, Fianna F\xe1il',
 17: u"O'Brien, Sinn F\xe9in",
 18: u'Gilroy, Labour Party',
 19: u'Browne, Fine Gael',
 20: u'Kelleher, Independent',
 21: u'Richmond, Fine Gael',
 22: u'Walsh, Progressive Democrats',
 23: u'Whelan, Labour Party',
 24: u'Crown, Fianna F\xe1il',
 25: u'Feighan, Fine Gael',
 26: u'Doherty, Sinn F\xe9in',
 27: u'O\u2019Brien, Independent',
 28: u'Buttimer, Fine Gael',
 29: u'McDowell, Labour Party',
 30: u'\xd3 Brolch\xe1in, Green Party',
 31: u'Brady, Fianna F\xe1il',
 32: u'Donohoe, Fine Gael',
 33: u'Higgin

In [38]:
# this gives us how many representations:
len(id_to_displayname)

199

In [39]:
# we again need to write down the metadata
import csv

with open('data/congress/Ireland/senator_metadata.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    for key, value in sorted(id_to_displayname.items()):
        writer.writerow([value.encode('utf8')])

In [40]:
# finally we build our model
from keras.layers import concatenate
from keras.layers import Dense, Input, Flatten
from keras.layers import MaxPooling1D, Embedding

embedding_layer = Embedding(len(id_to_displayname), 100)

# train a 1D convnet with global maxpooling
voting = voting_input = Input(shape=(1,), dtype='int32')
voting = embedding_layer(voting)
voting = Dense(32, activation='relu')(voting)
voting = Dense(32, activation='relu')(voting)

sponsor = sponsor_input = Input(shape=(1,), dtype='int32')
sponsor = embedding_layer(sponsor)
sponsor = Dense(32, activation='relu')(sponsor)
sponsor = Dense(32, activation='relu')(sponsor)

cosponsor = cosponsor_input = Input(shape=(10,), dtype='int32')
cosponsor = embedding_layer(cosponsor)
cosponsor = MaxPooling1D(10)(cosponsor)
cosponsor = Dense(32, activation='relu')(cosponsor)
cosponsor = Dense(32, activation='relu')(cosponsor)

combined = concatenate([voting, sponsor, cosponsor])
combined = Dense(32, activation='relu')(combined)
combined = Dense(1, activation='sigmoid')(combined)

Using TensorFlow backend.


In data/congress/Ireland, launch TensorBoard:
    
> davids-air:Ireland dazconap$ tensorboard --logdir=senator_reps/

In [41]:
from keras.models import Model
from keras.callbacks import TensorBoard

model = Model([voting_input, sponsor_input, cosponsor_input], combined)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

embedding_metadata = {
    embedding_layer.name: '../senator_metadata.csv'
}

model.fit([x_1, x_2, x_3], np.array(y).reshape(-1, 1, 1),
          batch_size=128,
          epochs=10,
          validation_split=0.2,
          callbacks=[TensorBoard(log_dir='data/congress/Ireland/senator_reps', 
                                 embeddings_freq=1,
                                 embeddings_metadata=embedding_metadata)])

Train on 30776 samples, validate on 7695 samples
Epoch 1/10
30776/30776 [==============================] - 3s 99us/step - loss: 0.0258 - acc: 1.0000 - val_loss: 2.5825e-07 - val_acc: 1.0000
Epoch 2/10
30776/30776 [==============================] - 3s 85us/step - loss: 1.1933e-07 - acc: 1.0000 - val_loss: 1.2733e-07 - val_acc: 1.0000
Epoch 3/10
30776/30776 [==============================] - 3s 82us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.2379e-07 - val_acc: 1.0000
Epoch 4/10
30776/30776 [==============================] - 3s 92us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.2287e-07 - val_acc: 1.0000
Epoch 5/10
30776/30776 [==============================] - 3s 82us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.2164e-07 - val_acc: 1.0000
Epoch 6/10
30776/30776 [==============================] - 3s 82us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.2124e-07 - val_acc: 1.0000
Epoch 7/10
30776/30776 [==============================] - 2s 75us/step - loss: 1.1921

Go to TensorBoard:
> http://localhost:6006/#projector
        
In TensorBoard, we can look at the representations created in our model using t-SNE or PCA. A t-SNE analysis using more than a 1.1K iterations divides the senators in three different groups:

![title](figures/t-SNE-Ireland.png)

### Utils

In [42]:
print '\xc3\xa1' == str('á')
print str('á').decode('utf-8').encode('utf-8')
print str('á').decode('utf-8').encode('utf-8') == str('\xc3\xa1')
print str('Tá').decode('utf-8').encode('utf-8') == str('T\xc3\xa1')
print str('Níl').decode('utf-8').encode('utf-8') == str('N\xc3\xadl')

True
á
True
True
True
